# Classes in Python
...and a bit in general, for those of you who haven't done any object-oriented programming yet.

Lars Tiede

## What is a class?

Many things :-/

There are several ways to explain classes, some rather abstract and philosophical, others more practical.

We don't have much time, so I go for *very* practical with a few examples.

## Classes by example: the class of pets

A class is a description of data ("attributes") and behavior ("methods") of objects that are "instances" of it:

In [1]:
class Pet:
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        self.pets_met = set()
        
    def advance_time(self):
        self.age += 1
        self.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def meet(self, other_pet):
        print("{} meets {}".format(self.name, other_pet.name))
        self.pets_met.add(other_pet)
        other_pet.pets_met.add(self)
        
    def print_stats(self):
        print("{o.name}, age {o.age}, hunger {o.hunger}, met {n} others".
             format(o = self, n = len(self.pets_met)))

my_degu = Pet("Vitali")

This is how we can read the above code:

* `my_degu` is an object: an instance of `Pet` (short: "my_degu is a pet").
* `Pet` itself can be understood in several ways: for example as "the concept of pet", or "the entirety of pets", or "the data type pet".
* `Pet` describes some properties and state of a pet (through the attributes `name`, `age`, `hunger`), and some behavior (the methods).
* Every method's first parameter is a reference to the object itself. By convention, it is called "self".
* `__init__` is a special method: the Python interpreter calls this method when a new Pet is created. The `__init__` method is called "the constructor" of the class.

Let's do something with pets:

In [2]:
my_degu.advance_time()

my_other_degu = Pet("Wladimir")
my_degu.meet(my_other_degu)

for _ in range(10):
    my_degu.advance_time()
    my_other_degu.advance_time()

for degu in [my_degu, my_other_degu]:
    if degu.hunger > 10:
        degu.feed()

my_degu.print_stats()
my_other_degu.print_stats()

Vitali meets Wladimir
Vitali, age 11, hunger 0, met 1 others
Wladimir, age 10, hunger 10, met 1 others


You see that you can use `my_degu` and `my_other_degu` like you have used any other objects before: by accessing attributes and methods usig "`.`", and by passing objects to functions as a value.

## Class attributes and class methods

A class can not only describe data and behavior of its instances, but also of itself ("the entirety of pets", or "the concept of pets").

We redefine the Pet class a bit:

In [3]:
class Pet:
    population = set()
    
    def __init__(self, name):
        self.name = name
        self.hunger = 0
        self.age = 0
        self.pets_met = set()
        self.__class__.population.add(self)
        
    def die(self):
        print("{} dies :(".format(self.name))
        self.__class__.population.remove(self)
        
    def is_alive(self):
        return self in self.__class__.population
        
    @classmethod
    def advance_time(cls):
        for pet in cls.population:
            pet.age += 1
            pet.hunger += 1
        
    def feed(self):
        self.hunger = 0
        
    def meet(self, other_pet):
        print("{} meets {}".format(self.name, other_pet.name))
        self.pets_met.add(other_pet)
        other_pet.pets_met.add(self)
        
    def print_stats(self):
        print("{o.name}, age {o.age}, hunger {o.hunger}, met {n} others".
             format(o = self, n = len(self.pets_met)))

This is how we can read what is new and exciting about our redefined class:

* `population` is a class attribute, i.e., it is defined on the class. `Pet.population` is the set of all pets. When a pet is created, it is added to the Pet population. When it `die()`s (a method we added), it is removed from the Pet population. For a given pet, we can check whether it's alive by calling `is_alive()`.
* Unlike `population`, the other attributes are instance attributes. `Pet.name` does not exist, but `my_other_degu.name` does.
* In instance methods like `__init__()` or `die()`, `self.__class__` refers to the class of self. We could just as well write `Pet` here, but it's better not to (mor on that later if we have time).
* `advance_time` is now a "class method". Its first parameter is not an instance, but the class itself. The parameter is usually called "cls".

In [4]:
my_degu = Pet("Vitali")
my_other_degu = Pet("Wladimir")

for _ in range(10):
    Pet.advance_time()

print("There are {} pets.".format(len(Pet.population)))
my_degu.die()
print("There are {} pets.".format(len(Pet.population)))
my_other_degu.die()
print("There are {} pets.".format(len(Pet.population)))

There are 2 pets.
Vitali dies :(
There are 1 pets.
Wladimir dies :(
There are 0 pets.


The only big change in the usage example compared to before is that instead of having to "advance time" for each pet individually, we now advance time for all pets in existence through our new class method.

## In Python, a class is an object

In Python, almost everything is an object. So is a class:

In [5]:
print(isinstance(Pet, object))
print(isinstance(my_degu, object))

True
True


This might look a bit confusing at first, but it really isn't. We have already used object semantics on `Pet`, and you probably did not even blink at it. The use of "`.`" to address an attribute, for instance, or the fact that we called a class method on the `Pet` class *just like* we called an instance method on a pet.

All you need to know is that you can shape a class and interact with a class in the same way as with any object. Like an object, a class has attributes holding data, and methods that you can call.

## Subclasses: the class of fishes

One thing you can do with classes is to *specialize* them by defining subclasses. For example, we can define a special kind of pet:

In [6]:
class Fish(Pet):
    def __init__(self, name, size):
        self.size = size
        super().__init__(name)
        
    def meet(self, other_fish):
        super().meet(other_fish)
        if not isinstance(other_fish, Fish):
            return
        if self.size > other_fish.size:
            self.feed()
            other_fish.die()
        elif self.size < other_fish.size:
            other_fish.feed()
            self.die()

Here's the rundown of what we did and what it means:

* `Fish` is a "subclass" of `Pet`. This means that "Fish is a Pet". Technically, it means that `Fish` "inherits" all attributes and methods from `Pet`. You'll soon see what that means exactly.
    * Nomenclature that is good to know: `Pet` is a "superclass" or "ancestor" of `Fish`. Since `Fish` is directly "derived from" `Pet`, `Pet` is also a "parent" or "base class" of `Fish`.
* `Fish.__init__()` looks a little different than `Pet.__init__()`. There is an extra parameter, and `Fish.__init__()` *calls* `Pet.__init__()` through `super()`.
* `super()` is the way to call a method in the parent class. It is a common pattern that a method that is "overridden" in a subclass (i.e. it exists in both subclass and superclass) calls `super()`. When `fish.f()` does the same as `pet.f()`, but..., then `fish.f()` probably calls `super().f()`.
* A fish can meet another fish, with predictable results.

Some code with fishes:

In [7]:
sushi = Fish("sushi", 1)
sharkie = Fish("sharkie", 5)

print("There are {} pets.".format(len(Pet.population)))
sushi.meet(sharkie)
print("There are {} pets.".format(len(Pet.population)))

There are 2 pets.
sushi meets sharkie
sushi dies :(
There are 1 pets.


### Subclasses inherit attributes and methods from their ancestors

`Fish` "inherits" all attributes and methods from `Pet`, which in turn inherits everything from `object` (implicitly).

That means that all attributes and methods of `Pet` and `object` are visible in `Fish`, unless `Fish` makes an own attribute or method of the same name.

That means that for example `Fish.population` is the same as `Pet.population`. "The same as" as in "literally the same thing"!

In [8]:
# id(obj) returns the memory address of obj
print(id(Fish.population), id(Pet.population))
print(id(Fish.population) == id(Pet.population))

140245423953736 140245423953736
True


Here's how that extends to methods (beware... technicalities ahead):

In [9]:
print(id(Fish.feed), id(Pet.feed))
print(id(Fish.feed) == id(Pet.feed))

140245316591824 140245316591824
True


Wait, wasn't `feed()` an instance method?

Yes, but it is *defined* and visible in the class, simply because we made `def feed(self)` in the scope of `class Pet`. However, Python *binds* it to an object when it is called on an object. Check this:

In [10]:
nemo = Fish("nemo", 1)

nemo.hunger = 10
nemo.feed()
print(nemo.hunger)

nemo.hunger = 10
Fish.feed(nemo)
print(nemo.hunger)

0
0


`nemo.feed()` and `Fish.feed(nemo)` do the same! That is because the first form is derived from the second form (more on that specific bit later).

Now you understand why every method inside a class has "`self`" as its first parameter, and why it is a good convention to name that parameter "`self`".

### Subclasses may override attributes and methods

`Fish` defines its own `__init__()` method. We say that `Fish` "overrides" `__init__()`. Consequently, these methods are not the same in `Fish` and `Pet`:

In [11]:
print(id(Fish.__init__), id(Pet.__init__))
print(id(Fish.__init__) == id(Pet.__init__))

140245317070920 140245317074456
False


## Objects inherit attributes from their class

Classes inherit and can override attributes from their parent classes.

Objects inherit and can override attributes from their class.

In [12]:
class A:
    def f(self):
        return self.__class__.__name__

class B(A):
    attr = 0

b = B()

print( B.f(b) ) # B inherits f from A
print( id(B.attr) == id(b.attr) ) # b inherits attr from B
b.attr = 1 # now b overrides attr
print( id(B.attr) == id(b.attr) )
print( B.attr, b.attr )

B
True
False
0 1


Fun fact about methods: instance methods just override the methods they inherit from the class. Remember:

In [13]:
B.f(b) == b.f() # b.f is just B.f "bound to b"

True

Effectively, the Python interpreter does something like this in `B.__init__()`:

```
self.f = functools.partial(B.f, self) TODO looks ugly on slide
```

... oh, just for fun and as a little reminder, nothing stops us from setting an object's attribute to something else:

In [14]:
b2 = B()
print( b2.f() )

def foo():
    return "foo"

b2.f = foo
print( b2.f() )

B
foo


**DON'T DO THIS AT HOME.**

## Attribute lookup in objects and classes

When you access `some_obj.some_attr`, this is the order of objects in which Python looks after `some_attr`:

1. `some_obj`
2. `some_obj.__class__`

When you access `SomeClass.some_attr`, this is the order of objects in which Python looks after `some_attr`:

1. `SomeClass`
2. ancestors of `SomeClass` in [C3 linearization](https://en.wikipedia.org/wiki/C3_linearization) order

This so-called "method resolution order" can for any class be looked up like so:

In [15]:
Fish.__mro__

(__main__.Fish, __main__.Pet, object)


## Magic methods (\__*methodname*\__)

Some methods on classes are called "magic methods". Their name usually starts and ends with two underscores.

Magic methods are methods that have a special meaning within Python. For example, `__init__` is the method that is called when an instance of a class is constructed.

There are many other magic methods, however. You can override them in your own classes. (They are defined in `object`, so you are overriding them.)

For example, `__repr__`, which is supposed to "return a canonical string representation of an object" (whatever that is). You can get this "canonical representation" for any object by calling `repr(obj)`:

In [16]:
print(repr(nemo))
print(object.__repr__(nemo))
print(nemo)

That string representation is not very readable for humans, especially when we do something like this:

In [17]:
print(Pet.population)

{<__main__.Fish object at 0x7f8d6846d080>, <__main__.Fish object at 0x7f8d684beef0>}


We can make this more readable by overriding `__repr__` in Pet. We do this guerilla-style here (you might not want to do this at home):

In [18]:
def better_pet_repr(o):
    return "{} ({})".format(o.name, o.__class__.__name__)
    
Pet.__repr__ = better_pet_repr

print(Pet.population)

{nemo (Fish), sharkie (Fish)}


## Interfaces

"Interfaces" are not a formal language construct in Python like they are in other languages, for example Java.

The informal meaning of "class C implements interface X" in Python is just "C implements the methods that an X has".

Example: an object `obj` "implements the 'callable' interface" when it implements `__call__()`. (Then you can call `obj()`.)

TODO code example

## Duck typing

> "If it quacks like a duck and looks like a duck, it's a duck."

In [19]:
class A:
    def quack(self):
        return "quackfoo"
    
class B:
    quack = lambda self: "quackbar" # saved a linebreak, woohoo!
    
def make_em_quack(*objs):
    for o in objs:
        print(o.quack())
    
make_em_quack(A(), B())

quackfoo
quackbar


The classes `A` and `B` are not related. They just both happen to have the method `quack()`. Which is all the function `make_em_quack()` needs. To that function, both `A` and `B` are sufficiently like a duck.

## `collections.abc`: make your own containers

The [collections.abc](https://docs.python.org/3.5/library/collections.abc.html) module gives us a lot of "base classes" we can subclass to easily make our own containers.

If we subclass such a "base class", we need only implement a few methods, and inherit the rest from the base class.

Example: `collections.abc.Mapping`
* you must supply only 3 methods: `__getitem__`, `__iter__`, and `__len__`
* you inherit all the rest that "makes a mapping": `__contains__`, `keys`, `items`, `values`, `get`, `__eq__`, and `__ne__`

In [20]:
from collections.abc import Mapping

class PetsByName(Mapping):
    def __iter__(self):
        return set(map(lambda p: p.name, Pet.population)).__iter__()

    def __getitem__(self, name):
        matches = set(filter(lambda p: p.name == name, Pet.population))
        if not matches:
            raise KeyError
        return matches
    
    def __len__(self):
        return sum(1 for _ in self.__iter__())

In [21]:
Pet.population = set()
f1 = Fish("Arielle", "blue")
f3 = Fish("Sushi", "silver")
p = Pet("Sushi")

m = PetsByName()
print( "There are {} names in the population.".format(len(m)) )
print( set(m.keys()) )
for name, pets in m.items():
    print("{}: {}".format(name, pets))

There are 2 names in the population.
{'Arielle', 'Sushi'}
Arielle: {Arielle (Fish)}
Sushi: {Sushi (Fish), Sushi (Pet)}


## Parting note: stop writing classes

Classes are cool and powerful, but tend to be used way too often when simpler, shorter "class-less" code would suffice.

*Homework: watch the famous [stop writing classes](https://www.youtube.com/watch?v=o9pEzgHorH0) talk.*

### So when write classes?

Writing a class might be a good idea when you model your problem with "lots of entities of mutable data, with functions dealing with such entities".